## Filter out remaining multiple hashtag tweets
When getting tweets from twitter query limited to 512 characters
Could only limit up to hashtag #chiefs for each query
Now need to create a filter for tweets collected for any tweets containing more than one team hashtag  

1. Figure out list of hashtags left
Ended on #chiefs for each query

In [93]:
from hashtags_all import hashtags
from hashtags_stripped import ht_flat, ht_half_1, ht_half_2
import pandas as pd


2. Read into dataframe tweets

In [94]:
# get team names into list
team_list = hashtags.keys()


In [96]:
test_df = pd.read_csv('./team_data/wk_12/wk_12_bears_tweets.csv')
test_df.head()

,team,timestamp,tweet_text,userid
0,bears,2022-11-27 16:58:58+00:00,"b'For the rest of the season, lol. #DaBears #B...",718550336575815680
1,bears,2022-11-27 16:58:31+00:00,b'\xf0\x9f\x9a\xa8BREAKING NEWS: Justin Fields...,1100098670324838403
2,bears,2022-11-27 16:57:45+00:00,b'#Jets RB James Robinson is a healthy inactiv...,1395552062650437635
3,bears,2022-11-27 16:55:18+00:00,b'AdamRonis: Justin Fields is inactive for the...,11221412
4,bears,2022-11-27 16:54:32+00:00,"b'Now- my subscription is 7,77 cause I like it...",1239072082077667328


3. Filter out current team hashtags from list of all hashtags left to filter
Should be left last half hashtags - team hashtags

In [97]:
team_name = 'bears'
def get_rest_hashtags(team_hashtags):
    output = [b for b in ht_half_2 if all(a not in b for a in team_hashtags)]
    return output

def list_to_lowercase(list):
    return [x.lower() for x in list]


def get_non_team_hashtags_to_negate(team_name, hashtags):
    team_hashtags = hashtags[str(team_name)]
    print(team_hashtags)
    non_team_hashtags = get_rest_hashtags(team_hashtags)
    non_team_hashtags_lowercase = list_to_lowercase(non_team_hashtags)
    return non_team_hashtags_lowercase

In [98]:
def df_column_to_lowercase(df, column):
    df[column].apply(lambda x: x.lower())
    return df

def filter_rows_df(df, hashtags):
    return df[~df['tweet_text'].str.contains('|'.join(hashtags))]

# need to deal with lowercase - make all tweets lowercase then do search with lowercase hashtags

In [99]:

def get_cleaned_df(filename, wk):
    # get info
    df = pd.read_csv(f'./team_data/wk_{wk}/{filename}')
    # get team name
    team_name = df.team[0]
    print(team_name)
    # get other team hashtags
    negate_hashtags = get_non_team_hashtags_to_negate(team_name, hashtags)
    # convert df tweets to lowercase
    df_lc = df_column_to_lowercase(df, 'tweet_text')
    
    # new df with only tweets with team hashtags
    clean_df = filter_rows_df(df, negate_hashtags)

    # inital length
    init_row_length = len(df.index)
    new_row_length = len(clean_df.index)
    num_rows_removed = init_row_length - new_row_length
    # get rows removed
    print(str(num_rows_removed), 'rows removed \t', new_row_length, ' remaining')
    # create info about rows removed
    info = [team_name, wk, init_row_length, num_rows_removed, new_row_length]
    
    return clean_df, team_name, info

In [91]:
def get_cleaned_df_2(df, wk):
    # get team name
    team_name = df.team[0]
    print(team_name)
    # get other team hashtags
    negate_hashtags = get_non_team_hashtags_to_negate(team_name, hashtags)
    
    # convert df tweets to lowercase
    df_lc = df_column_to_lowercase(df, 'tweet_text')

    # new df with only tweets with team hashtags
    clean_df = filter_rows_df(df_lc, negate_hashtags)


    # inital length
    init_row_length = len(df.index)
    new_row_length = len(clean_df.index)
    num_rows_removed = init_row_length - new_row_length
    # get rows removed
    print(str(num_rows_removed), 'rows removed \t', new_row_length, ' remaining')
    # create info about rows removed
    info = [team_name, wk, init_row_length, num_rows_removed, new_row_length]
    
    return clean_df, team_name, info

### get file names for each week folder

In [101]:
import os 
def iterate_through_files(week):
    path = f'./team_data/wk_{week}'
    results = os.listdir(path)
    return results

iterate_through_files('12')[:3]

['wk_12_49ers_tweets.csv',
 'wk_12_bears_tweets.csv',
 'wk_12_bengals_tweets.csv']

### Run through folder and filter hashtags for each file
1. get filenames from folder by week number [12:18]
2. for each file name get dataframe
3. filter by hashtags for each dataframe
4. Return new df, team name, info
5. Put info into array (info about how many tweets were removed)
6. Save new dataframe to csv file
7. Save info data to csv file

In [106]:
import csv 

def clean_all_files(week):
    file_paths = iterate_through_files(week)
    
    cleaning_info = []
    for path in file_paths:
        df,team, info = get_cleaned_df(path, week)
        cleaning_info.append(info)
        new_path = f'./team_tweets/wk_{week}/wk_{week}_{team}_hash_filtered.csv'
        df.to_csv(new_path)
    with open('cleaning_info_results.csv', 'a') as f:
        write = csv.writer(f)
        # write.writerow(['team','week','rows_initial','rows_removed','rows_remaining'])
        write.writerows(cleaning_info)


In [113]:
# week numbers
weeks = ['12', '13', '14','15','16','17','18']

clean_all_files(18)


49ers
['#FTTB', '#49ers']
17 rows removed 	 3188  remaining
bears
['#DaBears', '#BearsNation', '#Bears']
24 rows removed 	 3091  remaining
bengals
['#RuleTheJungle', '#Bengals', '#WhoDey']
8 rows removed 	 5537  remaining
bills
['#billsmafia', '#gobills', '#bills']
61 rows removed 	 7932  remaining
broncos
['#BroncosCountry', '#LetsRide', '#Broncos']
24 rows removed 	 2587  remaining
browns
['#Browns', '#ClevelandBrowns ', '#DawgPound']
25 rows removed 	 4275  remaining
buccaneers
['#GoBucs', '#bucs', '#buccaneers']
3 rows removed 	 1458  remaining
cardinals
['#BirdCityFootball', '#cardinals', '#AZCardinals']
109 rows removed 	 949  remaining
chargers
['#BoltUp', '#Chargers']
9 rows removed 	 1555  remaining
chiefs
['#ChiefsKingdom', '#Chiefs']
42 rows removed 	 4684  remaining
colts
['#ForTheShoe', '#Colts']
9 rows removed 	 1294  remaining
commanders
['#commanders', 'httc']
192 rows removed 	 1869  remaining
cowboys
['#DallasCowboys', '#Cowboys']
39 rows removed 	 2954  remaining
dol